In [32]:
%run Functions_Energy_Model.py

In [33]:
import yaml
import os

from Functions_Energy_Model import (
                                    location_italian_to_english,
                                    create_coordinates_dataset,
                                    suppress_printing,
                                    simulate_1_kWp_generators,
                                    simulate_gens_derated_productivity,
                                    simulate_unstacked_productivity,
                                    ) 

from Functions_General import (add_to_file_yml, generate_calendar)

---
---

## **Inputs**


Before to run the simulation we need to set some important parameter such as:

- **location_ita**: the name of the location in Italian;

- **tilt_angle**: the tilt angle of the module surface (inclination);

- **azimuth**: the azimuth angle of the module surface (south : 180; north: 0; east: 90; west: 270);

- **derating_factor**: derating factor for the yearly productivity of the photovoltaic generator in percentage.

In [34]:
location_ita = 'Milano' # location in Italian

In [35]:
capacity = 100 # kWp
tilt_angle = 40 # degrees
azimuth = 180 # degrees

In [36]:
num_years = 1 # number of years to simulate

In [37]:
derating_factor_percent = 1 # derating factor that reduce the efficiency of the modules in percentage [%]

In [ ]:
config = yaml.safe_load(open("config.yml", 'r'))
path_export = str(config['filename_output_csv_gen_pv']) # path to save the results

---
---

### **1. Initialize parameters for simulation**

Using the input parametes specified in the previous section we initialize the productivity simulation.

In [39]:
location_eng = location_italian_to_english(location_ita)

path = "config.yml"
filename = ""
key  = "project_lifetime_yrs"
value = num_years
add_to_file_yml(path, filename, key, value)

suppress_printing(generate_calendar) # generate the calendar for the location

coordinates_dataset = suppress_printing(create_coordinates_dataset, [location_eng]) # create a dataset with the coordinates of all the locations

derating_factor = derating_factor_percent / 100

### **2. Calculate productivity for 1 kWp**

For the selected location, a dictionary with the photovoltaic productivity of a generator with a capacity of 1 kWp is simulated.


The dictionary has a unique key equal to the location name in english and a dataframe as element. 

The dataframe has the following structure:

| datetime  (type DatetimeIndex)| 0|     
| :----:| :----:|
| 2025-01-01  00:00:00+01:00| 0| 
| 2025-01-01  00:15:00+01:00| 0| 
| ...| ...| ...| ...| ...| 

In [40]:
result_ac_energies_resampled = suppress_printing(simulate_1_kWp_generators, coordinates_dataset, tilt_angle, azimuth)

### **3. Scaling productivity to capacity of the generator**

The photovoltaic productivity is scaled respect the capacity of the generator.

In [41]:
result_ac_energies_gens = {} # initialization of the output dictionary
result_ac_energies_gens['gen_pv_' + str(capacity) + '_kWp'] = result_ac_energies_resampled[location_eng] * capacity

### **4. Derating productivity over years of the simulation**

The photovoltaic productivity is derated with a yearly derating factor.

The output is a dictionary with two level: the key for the first level is *gen_pv_{capacity}_kWp*, while in the second level the keys are the years of the simulation (e.g. 2025, 2026, ...). 

The element for each year is a dataframe with the following structure:

| datetime  (type DatetimeIndex)| 0|     
| :----:| :----:|
| 2025-01-01  00:00:00+01:00| 0| 
| 2025-01-01  00:15:00+01:00| 0| 
| ...| ...| ...| ...| ...| 

In [42]:
result_ac_energies_gens_derated = suppress_printing(simulate_gens_derated_productivity, derating_factor, result_ac_energies_gens)

### **5. Create a dataframe with results**

The dictionary is transformed in a dataframe with the following structure:

| datetime  (type DatetimeIndex)| gen_pv_{capacity}_kWp|     
| :----:| :----:|
| 2025-01-01  00:00:00| 0| 
| 2025-01-01  00:15:00| 0| 
| ...| ...| ...| ...| ...| 

In [43]:
result_ac_energies_to_csv_df = suppress_printing(simulate_unstacked_productivity, result_ac_energies_gens_derated) # create two unstacked dataframe (the other functions work with dictionaries)

### **6. Export results to csv file**

The dataframe is exported in a csv file.

The default folder for the exportation is:

        📂  *files/gen_pv*

In [ ]:
result_ac_energies_to_csv_df.to_csv(path_export, encoding='utf-8')

### **7. Open csv file**

In [ ]:
os.startfile(path_export)

---